In [1]:
%reload_ext autoreload
%autoreload 2

import cv2

import os
import sys
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import DataManager

from collections import defaultdict

from matplotlib.path import Path
%matplotlib inline

In [6]:
import numpy as np

In [3]:
stack = 'MD589'

In [4]:
sec = 195

In [7]:
patch_size = 224
half_size = patch_size/2
stride = 56

dm = DataManager(stack=stack, section=sec, load_mask=False)

w = dm.image_width
h = dm.image_height 

ys, xs = np.meshgrid(np.arange(half_size, h-half_size, stride), np.arange(half_size, w-half_size, stride),
                     indexing='xy')
    
sample_locations = np.c_[xs.flat, ys.flat]

In [ ]:
# # Whole slide

# output_dir = '/home/yuncong/CSHL_data_patches/%s/%04d/%s'%(stack, sec, label)
# if not os.path.exists(output_dir):
#     os.makedirs(output_dir)

# for i in indices_fg:
#     x, y = sample_locations[i]
# #             patch = dm.image_rgb[y-half_size:y+half_size, x-half_size:x+half_size]
#     patch = dm.image_rgb_jpg[y-half_size:y+half_size, x-half_size:x+half_size]
#     cv2.imwrite(output_dir + '/%s_%04d_%08d.jpg'%(stack, sec, i), img_as_ubyte(patch[..., ::-1]))

In [8]:
patch_dir = '/home/yuncong/CSHL_data_patches/'

In [9]:
box_x = 37
box_y = 122
box_w = 393
box_h = 219
xmin = box_x*32
xmax = (box_x+box_w)*32
ymin = box_y*32
ymax = (box_y+box_h)*32

In [10]:
mask = dm.load_thumbnail_mask()
indices_fg = np.where(mask[sample_locations[:,1]/32, sample_locations[:,0]/32])[0]
indices_bg = np.setdiff1d(range(sample_locations.shape[0]), indices_fg)

indices_roi = np.where(np.all(np.c_[sample_locations[:,0] > xmin, sample_locations[:,1] > ymin, 
                                    sample_locations[:,0] < xmax, sample_locations[:,1] < ymax], axis=1))[0]

indices_roi = np.setdiff1d(indices_roi, indices_bg)
print len(indices_roi), 'patches in ROI'

In [13]:
output_dir = os.path.join(patch_dir, '%s_byROI/%04d/roi1'%(stack, sec))
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [16]:
# dm._load_image(['rgb'])
dm._load_image(['rgb-jpg'])

for i in np.random.choice(indices_roi, min(10, len(indices_roi)), replace=False):
    x, y = sample_locations[i]
    patch = dm.image_rgb_jpg[y-half_size:y+half_size, x-half_size:x+half_size]
    cv2.imwrite(os.path.join(output_dir, '%(stack)s_%(sec)04d_%(gridp_ind)08d.jpg' %\
                             {'stack':stack, 'sec':sec, 'gridp_ind':i}), 
                patch[..., ::-1])

22401 patches


/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/PIL/Image.py:2224: DecompressionBombWarning: Image size (157463552 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


In [15]:
tarball_filename = '%(stack)s_%(sec)04d_roi1.tar' % {'stack': stack, 'sec': sec}
print 'tar -I pigz -C %(output_dir)s -cf %(tarball_filename)s .' % \
    {'output_dir': output_dir, 'tarball_filename': tarball_filename}

tar -I pigz -C /home/yuncong/CSHL_data_patches/MD589_byROI/0195/roi1 -cf MD589_0195_roi1.tar .
